In [1]:
import numpy as np
import pandas as pd
import cvxopt
from cvxopt import matrix, solvers
import os
import cv2

In [2]:
class_paths = ['./svm/train/0', './svm/train/1', './svm/train/2', 
               './svm/train/3', './svm/train/4', './svm/train/5']
images = [[] for i in range(6)]
for i  in range(len(class_paths)):
    for filename in os.listdir(class_paths[i]):
        img = cv2.imread(os.path.join(class_paths[i], filename))
        if img is not None:
            resized_img = cv2.resize(img, (16, 16))
            normalized_img = resized_img / 255.0
            flattened_img = normalized_img.flatten()
            images[i].append(flattened_img)

In [3]:
validation_paths = ['./svm/val/0', './svm/val/1', './svm/val/2', 
                    './svm/val/3', './svm/val/4', './svm/val/5']
validation_images = [[] for i in range(6)]
for i  in range(len(validation_paths)):
    for filename in os.listdir(validation_paths[i]):
        img = cv2.imread(os.path.join(validation_paths[i], filename))
        if img is not None:
            resized_img = cv2.resize(img, (16, 16))
            normalized_img = resized_img / 255.0
            flattened_img = normalized_img.flatten()
            validation_images[i].append(flattened_img)

In [4]:
def gaussian_matrix(X, gamma=0.001):
    squared_distances = np.sum(X**2, axis=1, keepdims=True) - 2 * np.dot(X, X.T) + np.sum(X**2, axis=1, keepdims=True).T
    kernel_matrix = np.exp(-gamma * squared_distances)
    return kernel_matrix

In [5]:
class Gaussian_SVM:

    def __init__(self, X, Y, C, gamma):
        self.C = C
        self.gamma = gamma
        self.X = X
        self.Y = Y

    def train(self):
        X = self.X
        Y = self.Y
        self.m = X.shape[0]
        self.n = X[0].shape[0]
        H = gaussian_matrix(X, gamma=self.gamma)
        P = matrix(H * (Y[:, np.newaxis] * Y), tc='d')
        q = matrix(-np.ones((self.m, 1)), tc='d')
        G = matrix(np.vstack((np.eye(self.m)*-1,np.eye(self.m))), tc='d')
        h = matrix(np.hstack((np.zeros(self.m), np.ones(self.m) * 1)), tc='d')
        A = matrix(Y.reshape(1, -1), tc='d')
        b = matrix(np.zeros(1), tc='d')
        sol = solvers.qp(P, q, G, h, A, b)
        self.alphas = np.array(sol['x'])
        self.num_sv = np.sum(self.alphas > 1e-5)
        self.support_vectors = []
        self.y_sv = []
        self.alpha_sv = []
        for i in range(self.m):
            if(self.alphas[i] > 1e-5):
                self.support_vectors.append(X[i])
                self.alpha_sv.append(self.alphas[i])
                self.y_sv.append(Y[i])
        self.support_vectors = np.array(self.support_vectors)
        self.alpha_sv = np.array(self.alpha_sv)
        self.y_sv = np.array(self.y_sv)
        kernel_matrix_bias = gaussian_matrix(self.support_vectors)
        new_matrix = kernel_matrix_bias * self.alpha_sv.reshape(-1, 1) * self.y_sv.reshape(-1, 1)
        bias_list = self.y_sv - np.sum(new_matrix, axis=0)
        self.b = 0
        num = 0
        for i in range(len(bias_list)):
            if(self.alpha_sv[i] < 1e-5 or (1 - self.alpha_sv[i]) < 1e-5): continue
            self.b += bias_list[i]
            num += 1
        self.b /= num
        self.b

    def prediction(self, x):
        prediction =  np.sum(np.exp(-0.001*np.sum((self.support_vectors - x)**2, axis = 1)) * self.y_sv * self.alpha_sv.reshape(self.alpha_sv.shape[0], ))  + self.b
        return prediction
    
    def predict_class(self, x):
        prediction = self.prediction(x)
        return 1 if prediction>=0 else -1

In [6]:
multi_class_svms = [[None for j in range(6)] for i in range(6)]
for i in range(6):
    for j in range(i + 1, 6):
        print(f"Starting SVM for classes {i}, {j}")
        X = np.vstack((np.array(images[i]), np.array(images[j])))
        Y = np.hstack((np.ones(len(images[i])), -np.ones(len(images[j]))))
        svm = Gaussian_SVM(X, Y, 1, 0.001)
        svm.train()
        multi_class_svms[i][j] = svm
        print(f"SVM trained for classes {i}, {j}")

Starting SVM for classes 0, 1
     pcost       dcost       gap    pres   dres
 0: -1.9393e+03 -1.4696e+04  9e+04  4e+00  3e-13
 1: -1.2731e+03 -1.0480e+04  2e+04  4e-01  3e-13
 2: -1.1845e+03 -2.6743e+03  1e+03  8e-13  2e-13
 3: -1.4357e+03 -2.1032e+03  7e+02  3e-12  2e-13
 4: -1.5025e+03 -1.9941e+03  5e+02  1e-13  2e-13
 5: -1.5733e+03 -1.8712e+03  3e+02  3e-13  2e-13
 6: -1.6192e+03 -1.7957e+03  2e+02  7e-13  2e-13
 7: -1.6452e+03 -1.7549e+03  1e+02  3e-13  2e-13
 8: -1.6661e+03 -1.7225e+03  6e+01  7e-14  2e-13
 9: -1.6769e+03 -1.7068e+03  3e+01  7e-13  2e-13
10: -1.6857e+03 -1.6947e+03  9e+00  5e-13  2e-13
11: -1.6890e+03 -1.6904e+03  1e+00  1e-13  3e-13
12: -1.6896e+03 -1.6897e+03  7e-02  5e-13  3e-13
13: -1.6896e+03 -1.6896e+03  2e-03  9e-13  3e-13
14: -1.6896e+03 -1.6896e+03  3e-05  4e-13  3e-13
Optimal solution found.
SVM trained for classes 0, 1
Starting SVM for classes 0, 2
     pcost       dcost       gap    pres   dres
 0: -2.8766e+03 -1.4989e+04  8e+04  3e+00  3e-13
 1: -1.

In [7]:
def multiclass_predict(x):
    distances = np.zeros((6,))
    votes = np.zeros((6,))
    for i in range(len(multi_class_svms)):
        for j in range(len(multi_class_svms)):
            if(multi_class_svms[i][j] is None): continue
            distance = multi_class_svms[i][j].prediction(x)
            if(distance >= 0): 
                votes[i] += 1
                distances[i] += abs(distance)
            else: 
                votes[j] += 1
                distances[j] += abs(distance)
    max_vote_index = np.argmax(votes)
    equal_vote_indices = np.where(votes == votes[max_vote_index])[0]
    if(len(equal_vote_indices) == 1): return max_vote_index
    else:
        max_distance_index = equal_vote_indices[np.argmax(distances[equal_vote_indices])]
        return max_distance_index

In [27]:
valid_correct = 0
valid_incorrect = 0
misclassified = []
misclassified_class = [[] for i in range(6)]
misclassified_prediction = [[] for i in range(6)]
confusion_matrix = np.zeros((6, 6))
for i in range(len(validation_images)):
    print(f"Starting validation for class {i}")
    j = 0
    for image in validation_images[i]:
        print(j, end=' ')
        prediction = multiclass_predict(image)
        confusion_matrix[i][prediction] += 1
        if(prediction == i): valid_correct += 1
        else: 
            valid_incorrect += 1
            misclassified.append(img)
            misclassified_class[i].append(j)
            misclassified_prediction[i].append(prediction)
        j += 1
    print()

Starting validation for class 0
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 
Starting validation for class 1
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 

In [28]:
print(f"Correct: {valid_correct}\nIncorrect: {valid_incorrect}\nAccuracy: {valid_correct / (valid_correct + valid_incorrect)}")
print(f"Confusion Matrix:\n{confusion_matrix}")

Correct: 667
Incorrect: 533
Accuracy: 0.5558333333333333
Confusion Matrix:
[[ 73.  22.  18.  27.  26.  34.]
 [  4. 150.   1.   6.  12.  27.]
 [  9.   4. 123.  26.  23.  15.]
 [ 23.   6.  25. 126.  15.   5.]
 [ 18.  17.  57.  33.  69.   6.]
 [ 25.  23.  10.   8.   8. 126.]]


In [46]:
actual = 1
labelled = 5
directory_images = os.listdir(f"./svm/val/{actual}/")
misclassed = []
for i in range(len(misclassified_class[actual])):
    if(misclassified_prediction[actual][i] == labelled): misclassed.append(directory_images[misclassified_class[actual][i]])